In [10]:
import requests
from bs4 import BeautifulSoup
from Bio import Entrez
import time
import os


In [28]:
def download_pdf_with_retry(url, file_path, headers=None, num_retries=3):
    retries = 0
    success = False
    while retries < num_retries and not success:
        try:
            response = requests.get(url, headers=headers)
            response.raise_for_status()  # 如果响应状态码不是200，将引发HTTPError异常
            with open(file_path, 'wb') as f:
                f.write(response.content)
            success = True
        except requests.HTTPError as e:
            print(f"Attempt {retries + 1} failed for {url}: {e}")
            retries += 1
            time.sleep(5)  # 在重试之间稍微延迟一下，避免连续请求

def search_pubmed_and_download_pdfs(email, term, download_folder="pdfs"):
    Entrez.email = email  # 请将其替换为您自己的电子邮件地址
    user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.71 Safari/537.36"
    headers = {"User-Agent": user_agent}

    if not os.path.exists(download_folder):
        os.makedirs(download_folder)

    with requests.Session() as session:  # 使用会话保持某些状态（例如cookies）
        session.headers.update(headers)

        # 使用Entrez搜索PubMed条目
        handle = Entrez.esearch(db="pubmed", term=term, retmax=200000)
        record = Entrez.read(handle)
        handle.close()

        for pubmed_id in record["IdList"]:
            try:
                # 尝试获取PMC ID，以构造PDF URL
                handle = Entrez.elink(dbfrom="pubmed", id=pubmed_id, linkname="pubmed_pmc")
                record = Entrez.read(handle)
                handle.close()

                pmc_id_links = record[0]["LinkSetDb"]
                if pmc_id_links and 'Link' in pmc_id_links[0]:
                    pmc_id = pmc_id_links[0]["Link"][0]["Id"]
                    pdf_url = f"https://www.ncbi.nlm.nih.gov/pmc/articles/PMC{pmc_id}/pdf/"
                    file_path = os.path.join(download_folder, f"PMC{pmc_id}.pdf")

                    download_pdf_with_retry(pdf_url, file_path, headers=headers, num_retries=3)
                else:
                    print(f"No PMC article available for PubMed ID: {pubmed_id}")
            except Exception as e:
                print(f"Error occurred for PubMed ID {pubmed_id}: {e}")

In [29]:
# 替换以下字段，并运行函数
your_email = "shou@nd.edu"  # 请用您的邮箱地址替换
search_term = "hypoxia, cell proliferation"  # 请输入您的搜索关键词
search_pubmed_and_download_pdfs(your_email, search_term)


No PMC article available for PubMed ID: 37894821
No PMC article available for PubMed ID: 37892195
No PMC article available for PubMed ID: 37892173
No PMC article available for PubMed ID: 37892142
No PMC article available for PubMed ID: 37890327
No PMC article available for PubMed ID: 37886854
No PMC article available for PubMed ID: 37885806
No PMC article available for PubMed ID: 37879890
No PMC article available for PubMed ID: 37879247
No PMC article available for PubMed ID: 37875691
No PMC article available for PubMed ID: 37874330
No PMC article available for PubMed ID: 37867353
No PMC article available for PubMed ID: 37865955
No PMC article available for PubMed ID: 37861358
No PMC article available for PubMed ID: 37859604
No PMC article available for PubMed ID: 37858471
No PMC article available for PubMed ID: 37857014
No PMC article available for PubMed ID: 37856073
No PMC article available for PubMed ID: 37845450
No PMC article available for PubMed ID: 37844208
No PMC article avail